# Dask

La bază, dask este o librărie în Python care ne ajută să facem scale la datele cu care lucrăm și să creștem performanța. Să începem cu ceva cod pentru a vedea cum anume funcționează dask

In [1]:
from dask.distributed import Client
client = Client(n_workers=2, threads_per_worker=1, memory_limit='1GB')

client

2022-08-12 08:04:55,342 - distributed.diskutils - INFO - Found stale lock file and directory '/var/folders/25/wbp849wd23g71mfg09jwp7g88qjfs0/T/dask-worker-space/worker-gnaxqu9u', purging
2022-08-12 08:04:55,343 - distributed.diskutils - INFO - Found stale lock file and directory '/var/folders/25/wbp849wd23g71mfg09jwp7g88qjfs0/T/dask-worker-space/worker-u2mptzbn', purging


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 2
Total threads: 2,Total memory: 1.86 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:54754,Workers: 2
Dashboard: http://127.0.0.1:8787/status,Total threads: 2
Started: Just now,Total memory: 1.86 GiB
Comm: tcp://127.0.0.1:54764,Total threads: 1
Dashboard: http://127.0.0.1:54766/status,Memory: 0.93 GiB
Nanny: tcp://127.0.0.1:54757,


Codul de mai sus nu este necesar de fiecară dată, dar să fim obișnuiți cu faptul că dask utilizează un scheduler distribuit pentru a împărți munca, fie că o împrate la diferite CPU-uri de pe un laptop sau la diferite clustere

Următorul lucru pe care o să îl facem este să importăm dask.dataframe. Aceste dataframe este ca și un sub API de pandas. Utilizând dask.dataframe este aproape identic cu utilizarea unui dataframe în Pandas. Majoritatea funcțiilor apelate în dask sunt la fel precum cele ce se apelează în pandas. O să lucrăm cu un set de date mai mic, cu un csv pentru a putea vedea ceva exemple din dask. Fișierul csv cu care lucrăm se găsește în folderul 'data'

Pentru început o să importăm dask.dataframe pentru a putea crea un dataframe cu aceste date

In [2]:
import dask.dataframe

Pentur a putea citi un fișier csv o să se utilizeze aceeași metodă ca și în pandas, și anume metoda 'read_csv()'. Pe lângă numele fișierului, în dasl mai putem să oferim un argument ce poartă denumirea de 'blocksize'

In [13]:
ddf = dask.dataframe.read_csv('./data/beer_small.csv', blocksize=10e6)

Acel argument de blocksize înseamnă că o să citim acel csv în bucăți în loc să îl citim pe tot deodată. Dacă se utiliza pandas, atunci citeam acest set de date în întregime în memorie, iar dacă acel data set era mai mare decât memoria mașinii puteam să primim o eroare de tipul 'Out of memory'. Utilizând dask putem să citim acel fișier în bucăți. Aceste bucăți poartă denumirea fie de 'chunks', fie de 'blocks' sau 'partitions'.

In [14]:
ddf

,Unnamed: 0,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
npartitions=2,,,,,,,,,,,,,,
,int64,int64,object,int64,float64,float64,float64,object,object,float64,float64,object,float64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...


Dacă afisăm acel data frame, putem să vedem structura lui. Putem observa faptul că avem numele coloanelor și tipul de date din fiecare coloană, însă datele din cadrul acelui dataframe lipsesc, însă putem vedea că avem informații adiționale. 'npartition=2' ne spune că citirea fișierului o să se facă în două partiții. Ce anume înseamnă informațiile de mai sus? Înseamnă că dask s-a uitat peste acel fișier csv, dar s-a uitat foarte puțin la el, s-a uitat la numele coloanelor și la tipul de date din fiecare coloană. Mai mult, dask a împărțit acel fișier în două partiții, în 2 bucăți. Ceea ce nu a făcut dask, nu a citit acele date din cadrul fișierului. Acest lucru este important deoarece dacă avem un fișier mare de câțiva GB poate nu dorim să citească fișierul doar dacă este nevoie de așa ceva

Fiecare dintre partiții reprezintă două metode separate de 'read_csv()'. Atunci când dorim să facem ceva operații cu acest fișier, doar atunci o să citim în întregime fișierul csv.

Ce anunme este un dask.dataframe?

<img src='./SS/01.png'>

Fiecare dintre partițiile prezente în Dask reprezintă la bază un dataframe din Pandas. Toate aceste pariții la final o să alcătuiască un dask dataframe. Dacă dorim să facem ceva operații cu acel fișier, dask o să îi spună lui pandas ce anume să facă și la final o să ne returneze rezutatele ca și un singur dataframe.

Există o metodă prin care putem apela o funcție pe fiecare dintre partițiile create în dask. Metoda respectivă poartă denumirea de 'map_partitions()'. Să utilizăm metoda 'type()' (de notat că această metodă nu se apelează, doar se trece ca și argument, deci fără paranteze) pentru fiecare dintre partiții

In [15]:
ddf.map_partitions(type)

Dask Series Structure:
npartitions=2
    object
       ...
       ...
dtype: object
Dask Name: type, 2 graph layers

Din nou, linia de cod de mai sus nu ne oferă nicio informație relevantă, ci doar ne o spune ce anume s-ar întâmpla dacă am rula acea comandă. Pentru a îi spune lui dask să facă o anumită operație, atunci trebuie să utilizăm metoda '.compute()'. Aceasta este cea mai des utilizată metodă prin care îi spunem lui dask să facă ceva anume

In [16]:
ddf.map_partitions(type).compute()

0    <class 'pandas.core.frame.DataFrame'>
1    <class 'pandas.core.frame.DataFrame'>
dtype: object

Deoarece metoda 'type' ne spune ce tip de date este un anumit obect, din moment ce am rulat această metodă pentru partițiile din dask (și fiindcă sunt prezente două partiții în dask), primim înapoi informația cum că în cadrul dask dataframe-ului avem 2 dataframe-uri de tip pandas, lucru care confirmă ideea că un dask dataframe este alcătuit din mai multe pandas dataframe-uri.

Putem să utilizăm pentru un dask dataframe operații de bază din pandas, cum ar fi utilizarea metodei 'head()'

In [17]:
ddf.head()

,Unnamed: 0,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,784200,952,Great Dane Pub & Brewing Company (Downtown),1136269921,4.5,4.0,4.0,dirtylou,American IPA,4.0,4.0,Texas Speedbump IPA,NaN,11846
1,1305265,29,Anheuser-Busch,1234830966,4.5,4.0,3.0,talkinghatrack,Light Lager,3.0,4.0,Bud Light Lime,4.2,41821
2,1526298,45,Brooklyn Brewery,1078599557,4.5,4.0,4.0,PopeJonPaul,Scotch Ale / Wee Heavy,4.0,4.5,Brooklyn Heavy Scotch Ale,7.5,16355
3,450647,590,New Glarus Brewing Company,1288790879,4.5,4.5,4.5,sweemzander,American Wild Ale,4.5,4.0,R&D Bourbon Barrel Kriek,5.5,60588
4,1223094,4,Allagash Brewing Company,1295320417,4.5,4.5,4.0,Jmoore50,American Wild Ale,4.0,4.0,Allagash Victor Francenstein,9.7,56665


Pentru metoda 'head()' nu mai trebuie să apelăm și 'compute()' deoarece dask face automat asta pentru noi. Citește primele 5 rânduri din prima partiție și le afișează. De asemenea putem și filtra aceste date dintr-un dask dataframe precum le-am utiliza utilizând un pandas dataframe

In [18]:
ddf[ddf['beer_style'].str.contains('IPA')].head()

,Unnamed: 0,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,784200,952,Great Dane Pub & Brewing Company (Downtown),1136269921,4.5,4.0,4.0,dirtylou,American IPA,4.0,4.0,Texas Speedbump IPA,NaN,11846
9,426580,666,Emerson's Brewery,1192461083,5.0,4.0,4.5,Lukie,English India Pale Ale (IPA),4.0,5.0,1812 India Pale Ale,4.7,4594
24,728901,17963,Nectar Ales,1312873910,3.5,4.0,3.5,Sensaray,American IPA,3.5,3.5,IPA Nectar,6.8,9024
26,745463,12877,NINE G Brewing Company,1189556274,4.0,4.5,4.0,Phatz,American Double / Imperial IPA,4.0,4.5,Infidel Imperial IPA,8.4,31041
28,94239,140,Sierra Nevada Brewing Co.,1269655771,4.0,4.5,4.5,CaptainIPA,American IPA,4.5,4.5,Sierra Nevada Torpedo Extra IPA,7.2,30420


Putem să salvăm acel output în cadrul unei variabile și putem să ne folosim de acea variabilă pentru a grupa elementele din dataframe

In [19]:
ipa = ddf[ddf['beer_style'].str.contains('IPA')]

In [20]:
mean_ipa_review = ipa.groupby('brewery_name').review_overall.agg(['mean', 'count'])

Linia de mai sus, din nou nu creează nimic, doar creează anumite task-uri pentru dask pe care le putem rula. Dacă afișăm ce anume se găsește în cadrul variabilei 'mean_ipa_review' o să vedem din nou informații generale

In [22]:
mean_ipa_review

,mean,count
npartitions=1,,
,float64,int64
,...,...


Din acea informație ne putem da seama că o să avem un nou dataframe cu două coloane și ce tip de date o să fie prezente în cadrul acelor coloane. De asemenea ne și spune câte task-uri are de rulat dask pentru a putea realiza această operație, iar informația aceasta se găsește în 'Dask Name: getitem, 7 graph layers'. Din acea informație dask ne spune că are de rulat 7 task-uri. Pentru a rula aceste task-uri, trebuie să apelăm metoda '.compute()'

In [23]:
mean_ipa_review.compute()

,mean,count
brewery_name,,
(512) Brewing Company,3.785714,7
1516 Brewing Company,4.000000,1
1702 / The Address Brewing Co.,4.000000,1
21st Amendment Brewery,3.923469,98
7 Seas Brewery and Taproom,4.000000,1
...,...,...
Yak and Yeti,4.500000,1
York Brewery Company Limited,4.000000,1
Yukon Brewing Company,4.250000,2


În continuare, putem realiza operații standard de pandas pentru acest dask dataframe

In [24]:
mean_ipa_review.nlargest(20, 'mean').compute()

,mean,count
brewery_name,,
Burnside Brewing Co.,5.000000,1
Elk Mountain Brewing,5.000000,1
Pioneer Brewing Co.,5.000000,2
Crouch Vale Brewery Limited,5.000000,1
Feral Brewing Co.,5.000000,1
Flour City Brewing Co.,5.000000,1
La Jolla Brew House,5.000000,1
Uncle Buck's Brewery & Steakhouse,5.000000,1
Glacier Brewhouse,4.875000,4


Având atât de multe date într-un fișier csv este ok momentan, putem lucra local cu acest fișier, dar în mod normal o să dorim să lucrăm cu un data set dristribuit într-un cluster. Dask are de făcut o mulțime de operații de citire și scriere din diferite tipuri de date (csv, hdf, json, parquet). Deoarece totul este împărțit în bucăți, în partiții, procedeele de scriere și de citire devin un pic mai complicate. Cel mai complicat este procedeul de scriere. În cazul de față avem 2 worker (putem avea și mii de workers) și dacă avem ceva date unde fiecare dintre workers o să aibă efectiv ceva de lucru și dorim la final să scriem ceva pe disk avem 2 posibilități. Fie agregăm toate acele date în memorie și scriem toate datele din memorie pe disk sau fie putem să facem acest lucru în paralel, unde fiecare worker își scrie datele într-un format distribuit.

La final, ceea ce este indicat să facem este să ne închidem clientul

In [25]:
client.close()